# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Torch imports
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models

# For plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Further imports
import numpy as np
from collections import OrderedDict
from PIL import Image
import json

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# Define transforms for the training, validation, and testing sets
data_transforms = {'train': transforms.Compose([transforms.Resize((250,250)),
                                      transforms.RandomCrop((224,224)),
                                      transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]),
                   'valid': transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]),
                   'test': transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]),
                  }



# Load the datasets with ImageFolder
image_datasets = {'train': datasets.ImageFolder(train_dir, transform = data_transforms['train']),
                  'valid': datasets.ImageFolder(train_dir, transform = data_transforms['valid']),
                  'test': datasets.ImageFolder(train_dir, transform = data_transforms['test'])
                 }

# Using the image datasets and the trainforms, define the dataloaders
dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size = 64, shuffle=True),
               'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size = 64, shuffle=True),
               'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size = 64, shuffle=True),
              }

In [ ]:
# Display example image
data_iter = iter(dataloaders['train'])

images = next(data_iter)


image = images[0][0].numpy().transpose((1, 2, 0))

fig, ax = plt.subplots()

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
image = std * image + mean
image = np.clip(image, 0, 1)

ax.imshow(image)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(axis='both', length=0)
ax.set_xticklabels('')
ax.set_yticklabels('')

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [ ]:
# Create model
model_name = 'densenet121'
model = eval("models." + model_name + "(pretrained=True)")
model = models.densenet121(pretrained=True)
model.epochs = 0
model

In [ ]:
# To prevent backprop through parameters freeze parameters
input_units = 1024
hidden_units = 512
prob_dropout = 0.3

for param in model.parameters():
    param.requires_grad = False
    
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(input_units, hidden_units)),
    ('relu1', nn.ReLU()),
    ('dropout1', nn.Dropout(prob_dropout)),
    ('fc2', nn.Linear(hidden_units,102)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.classifier = classifier

In [ ]:
def validation(model, loader, criterion, device):
    with torch.no_grad():
        accuracy = 0
        loss = 0
        for inputs, labels in loader:                            
            inputs, labels = inputs.to(device), labels.to(device)

            output = model.forward(inputs)
            loss += criterion(output, labels).item()

            ps = torch.exp(output)
            equality = (labels.data == ps.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()

        loss = loss/len(loader)
        accuracy = accuracy/len(loader)
      
    return loss, accuracy   

In [ ]:
def train_model(model, trainloader, criterion, optimizer, epochs = 3, print_every = 40, validloader=None):
    steps = 0
    
    
    model.train()
    # Train model on gpu if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
        
    for e in range(epochs):
        running_loss = 0
        for inputs, labels in trainloader:
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            output = model.forward(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if steps % print_every == 0:
                #print("calculate accuracy")
                
                model.eval()
                
                # Calculate loss and accuracy of validation set
                if validloader:
                    loss_val, accuracy_val = validation(model, validloader, criterion, device)                    
                        
                    print("Epoch: {}/{}\t".format(e+1,epochs),
                          ("Step: {}\t".format(steps)),
                          ("Loss (test): {:.4f}\t".format(running_loss/print_every)),
                         ("Loss (val): {:.4f}\t".format(loss_val)),
                         ("Accuracy (val): {:.4f}\n".format(accuracy_val)))
                else:
                    print("Epoch: {}/{}\t".format(e+1,epochs),
                          ("Loss: {:.4f}".format(running_loss/print_every)))
                running_loss = 0
                
                model.train()
        model.epochs += 1

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
epochs = 5

In [ ]:
train_model(model, dataloaders['train'], criterion, optimizer, epochs = epochs, print_every = 40, validloader = dataloaders['valid'])
#train_model(model, dataloaders['train'], criterion, optimizer, epochs = epochs, print_every = 10)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
# Make sure network is in eval mode for inference
model.eval()

# Turn off gradients for validation, saves memory and computations
with torch.no_grad():
    loss_test, accuracy_test = validation(model, dataloaders['valid'], criterion, 'cuda') 

print(("Loss (test): {:.4f}\t".format(loss_test)),
     ("Accuracy (test): {:.4f}\n".format(accuracy_test)))

model.train()

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Save model parameters
model.class_to_idx = image_datasets['train'].class_to_idx

model.name = model_name
model.input_units = input_units
model.hidden_units = hidden_units
model.prob_dropout = prob_dropout

In [ ]:
def save_model(model, optimizer):
    checkpoint = {'state_dict': model.state_dict(),
                  'class_to_idx': model.class_to_idx,
                  'name': model.name,
                  'input_units': model.input_units,
                  'hidden_units': model.hidden_units,
                  'prob_dropout': model.prob_dropout,
                  'n_epochs': model.epochs,
                  'optimizer' : optimizer.state_dict}

    torch.save(checkpoint, 'checkpoint.pth')

    print("model saved to checkpoint.pth")

In [ ]:
# TODO: Save the checkpoint 
save_model(model, optimizer)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
def load_checkpoint(filepath):
    """
    Function that loads a checkpoint and rebuilds the model
    """
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")    
    
    if device == "cuda:0":
        checkpoint = torch.load(filepath)
    else:
        checkpoint = torch.load(filepath, map_location={'cuda:0': 'cpu'})
        
    model = eval("models." + checkpoint['name'] + "(pretrained=True)")
    
    for param in model.parameters():
        param.requires_grad = False
        
    # Create the classifier
    classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(checkpoint['input_units'], checkpoint['hidden_units'])),
    ('relu1', nn.ReLU()),
    ('dropout1', nn.Dropout(checkpoint['prob_dropout'])),
    ('fc2', nn.Linear(checkpoint['hidden_units'],102)),
    ('output', nn.LogSoftmax(dim=1))
    ]))

    model.classifier = classifier
    
    #model.classifier = checkpoint['model_classifier']
    model.load_state_dict(checkpoint['state_dict'])
    
    model.epochs = checkpoint['n_epochs']    
    model.class_to_idx = checkpoint['class_to_idx']

    
    
    return model

In [ ]:
model_2 = load_checkpoint('checkpoint.pth')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Load image
    image = Image.open(image_path)
    
    # scale image
    size = 256, 256
    image.thumbnail(size)
    
    # center crop
    width, height = image.size   # Get dimensions
    new_width = 224
    new_height = 224
    left = (width - new_width)/2
    
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    image = image.crop((left, top, right, bottom))
    
    np_image = np.array(image)
    
    # Normalize
    means = np.array([0.485, 0.456, 0.406])
    stdev = np.array([0.229, 0.224, 0.225])
    
    np_image = (np_image/255 - means)/stdev
    
    image_transposed = np_image.transpose(2,0,1)
    
    # TODO: Process a PIL image for use in a PyTorch model
    return image_transposed

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # Load preprocessed image
    image = process_image(image_path)
    
    
    # Add additional "image index dimension"
    image = np.expand_dims(image,0)
    
    image_tensor = torch.FloatTensor(image)
    
    model.eval()
    
    with torch.no_grad():
    
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model.to(device)
        image_tensor = image_tensor.to(device)
        output = model.forward(image_tensor)
        
        ps = torch.exp(output)
        
        probs, probs_index = ps.topk(topk)
        
        probs, probs_index = probs.tolist()[0], probs_index.tolist()[0]
        
        class_to_idx = model.class_to_idx
        idx_to_class = {y:x for x,y in class_to_idx.items()}
        
        classes = [idx_to_class[x] for x in probs_index]      
    
    
    model.train()
    

    return probs, classes

In [ ]:
image_path = "flowers/test/14/image_06091.jpg"
image_class = 14
probs, classes = predict(image_path, model_2, topk=5)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# Initialize figure
image = process_image(image_path)
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(7,7))

# Plot image on axes1
imshow(image, ax[0])
ax[0].axis('off')
ax[0].set_title(cat_to_name[str(image_class)])
#ax[1].barh(probs)

# Plot barchart on axes2
# Sort values
probs_sorted = [probs for probs,_ in sorted(zip(probs,classes), reverse=False)]
classes_sorted = [classes for _,classes in sorted(zip(probs,classes), reverse=False)]
classes_sorted_name = [cat_to_name[str(x)] for x in classes_sorted]

y_pos = np.arange(len(classes_sorted_name))
ax[1].barh(y_pos, probs_sorted,tick_label = classes_sorted_name, align="center")